# Autolabel TMA Cores

## This notebook is an example: create a copy before running it or you will get merge conflicts!

**NOTE**: Before running this notebook for the first time, make sure you've coregistered your instrument using the *update coregistration parameters* section of  `1_set_up_toffy.ipynb`. This will ensure your FOVs display correctly on the slide.



### Background
This notebook automatically checks the names assigned to the cores on a TMA. In order to get the most benefit out of the notebook, make sure that you've named your FOVs appropriately. The expected format is RNCM, where N is the row and M is the column of the TMA. For example, a core on the third row and second column would be R3C2, and one on the 7th row and first column would be R7C1.  

The script expects that you have already generated and moved the necessary files into the appropriate directory before starting.
- A JSON file defining the four corners of the TMA. It's important that you have selected them in the correct order; top left, top right, bottom left, bottom right. Even if one of the cores on the corner is missing, make sure the FOV is located where that corner of the TMA *would* be located, as this is used to define the dimensions of the TMA. You can create this file by exporting the FOVs from the MIBIControl software. 
- A JSON file containing all of the FOVs that you have selected from the TMA, named appropriately. You can create this file by exporting the FOVs from the MIBIControl software. 
- The optical image of your TMA slide. This is automatically created when you load your slide, and is saved to the *Data/optical-image* subfolder

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import json
import os
from skimage.io import imread

from toffy import tiling_utils, json_utils
from toffy.json_utils import write_json_file

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

### 1. Copy over the necessary files to start the script

You will first need to define the prefix to use for all of the files associated with this specific TMA. The default is `tma_name`, but you should change it to something relevant to your study, such as BRCA_TMA_1.

Once you have picked your prefix for this specific TMA, you'll need to ensure that all of the necessary files are in the appropriate directory with the correct names

* `tma corners file`: this file, which contains the FOVs defining the four corners of the TMA, should be named `tma_name_corners.json`, where `tma_name` is replaced with the `tma_prefix`. 
* `manual run file`: this file, which contains the manually selected FOVs from your TMA, should be named `tma_name_manual.json`.
* `optical image file`: this file, which contains the image of your slide, should be named `tma_name.bmp`

Each of these files should be copied to `C:\\Users\\Customer.ION\\Documents\\autolabeled_tma_jsons`

In [ ]:
# define the prefix for each file
tma_prefix = 'example_tma'

In [ ]:
# user created files
tma_dir = os.path.join('C:\\Users\\Customer.ION\\Documents\\autolabeled_tma_jsons')
tma_corners_path = os.path.join(tma_dir, '%s_corners.json' % tma_prefix)
manual_run_path = os.path.join(tma_dir, '%s_manual.json' % tma_prefix)
slide_path = os.path.join(tma_dir, tma_prefix + '.bmp')

# files the notebook will create
auto_fov_names_path = os.path.join(tma_dir, '%s_automatic_fov_names.json' % tma_prefix)
mapping_path = os.path.join(tma_dir, '%s_mapping.json' % tma_prefix)
remapped_fov_path = os.path.join(tma_dir, '%s_automatic_run.json' % tma_prefix)
moly_path = os.path.join(tma_dir, '%s_moly_point.json' % tma_prefix)

### 2. Generate the automatic mapping of FOV names

In [ ]:
# Define TMA grid dimensions
tma_num_row = 7
tma_num_col = 4

# generate automatically named TMA
auto_fov_regions = tiling_utils.generate_tma_fov_list(
    tma_corners_path,
    tma_num_row,
    tma_num_col
)

# save the automatically-named TMA FOVs to centroids mapping
write_json_file(json_path=auto_fov_names_path, json_object=auto_fov_regions, encoding='utf-8')

In [ ]:
# load the user-defined set of FOVs in
with open(manual_run_path, 'r', encoding='utf-8') as mfop:
    manual_fov_regions = json.load(mfop)
    
# ensure missing and duplicate FOV names get identified
manual_fov_regions = json_utils.rename_missing_fovs(manual_fov_regions)
manual_fov_regions = json_utils.rename_duplicate_fovs(manual_fov_regions)

In [ ]:
# load the slide image in
slide_data = imread(slide_path)

### 3. Set thresholds for identifying incorrect FOV names

The variables below control the tolerance for identifying when a core has been named incorrectly.

* `check_dist`: set to a positive value to notify of FOV mappings at a distance greater than this value (measured in microns), sorted by decreasing distance. Set to `None` to bypass.
* `check_duplicates`: set to `True` to flag FOVs in `auto_fov_regions` with multiple FOVs mapping to it. Set to `False` to bypass.
* `check_mismatches`: set to `True` to flag FOVs with mismatched names. Set to `False` to bypass. Assumes FOVs have been named R1C1, R1C2, etc. 

In [ ]:
check_dist = 2000
check_duplicates = True
check_mismatches = True

Each FOV in `manual_fov_regions` are mapped to their closest corresponding FOV in `auto_fov_regions` by default. To see the current mappings, select FOVs in the `Manually-defined FOV` menu. To remap a manual FOV to a different auto FOV, use the `Automatically-generated FOV` menu.

After you're done finished, click `Save mapping` and run the cells afterward (ignore any error messages that may appear there beforehand). You can always come back here and redo your mappings if you change your mind.

In [ ]:
%matplotlib widget
tiling_utils.tma_interactive_remap(
    manual_fov_regions,
    auto_fov_regions,
    slide_data,
    mapping_path,
    draw_radius=7,
    figsize=(7, 7),
    check_dist=check_dist,
    check_duplicates=check_duplicates,
    check_mismatches=check_mismatches
)

### 4. Set parameters for created remapped JSON

The variables below will control how the remapped JSON is created

* `randomize`: shuffle the order of the FOVs in `remapped_fov_regions` to avoid potential batch effects of acquisition order
* `moly_insert`: insert a moly FOV between a specified interval of FOVs
* `moly_interval`: if `moly_insert` is set, controls how many FOVs are between each subsequent moly FOV

In [ ]:
randomize = True
moly_insert = True
moly_interval = 5

In [ ]:
# load the mapping saved by the interactive visualization
with open(mapping_path, 'r', encoding='utf-8') as mp:
    mapping = json.load(mp)

# rename FOVs, randomize the order, and insert moly points at a specified interval
remapped_fov_regions = tiling_utils.remap_and_reorder_fovs(
    manual_fov_regions,
    mapping,
    moly_path,
    randomize=randomize,
    moly_insert=moly_insert,
    moly_interval=moly_interval
)

# save remapped_fov_regions
write_json_file(json_path=remapped_fov_path, json_object=remapped_fov_regions, encoding='utf-8')